In [1]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=e8ac13713951b3ba79c59469c32c11ef45d1b3639c53d6627f35c89c8a92f90a
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [8]:
import sqlite3
import pandas as pd
import os
import pandasql
import random
from datetime import datetime, timedelta

In [9]:
# Helper function to generate random dates
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

# Date range for sales records
date_start = datetime(2022, 1, 1)
date_end = datetime(2023, 3, 31)

# Sample CUSTOMERS data
customers_data = {
    "customer_id": [f"CUST{i:03d}" for i in range(1, 21)],
    "first_name": ["John" if i % 4 == 0 else "Jane" for i in range(20)],  # Adding 'John' every 4 customers
    "last_name": ["Doe" if i % 4 == 0 else "Smith" for i in range(20)],   # Adding 'Doe' every 4 customers
    "address": [f"{random.randint(1, 100)} Main St, City {i}" for i in range(20)]
}
customers_df = pd.DataFrame(customers_data)
print("CUSTOMERS Dataset:")
print(customers_df)

# Sample ITEMS data
items_data = {
    "item_id": [f"ITEM{i:03d}" for i in range(1, 11)],
    "item_name": [f"Item {i}" for i in range(1, 11)],
    "price": [round(random.uniform(5, 100), 2) for _ in range(10)],
    "department": [random.choice(["Electronics", "Clothing", "Home", "Toys", "Books"]) for _ in range(10)]
}
items_df = pd.DataFrame(items_data)
print("\nITEMS Dataset:")
print(items_df)

# Sample SALES data
sales_data = {
    "date": [random_date(date_start, date_end).strftime("%Y-%m-%d") for _ in range(50)],
    "order_id": [f"ORD{i:03d}" for i in range(1, 51)],
    "item_id": [random.choice(items_df["item_id"]) for _ in range(50)],
    "customer_id": [random.choice(customers_df["customer_id"]) for _ in range(50)],
    "quantity": [random.randint(1, 5) for _ in range(50)]
}

# Calculate revenue for each sale
sales_data["revenue"] = [
    round(sales_data["quantity"][i] * items_df[items_df["item_id"] == sales_data["item_id"][i]]["price"].values[0], 2)
    for i in range(50)
]
sales_df = pd.DataFrame(sales_data)
print("\nSALES Dataset:")
print(sales_df)


CUSTOMERS Dataset:
   customer_id first_name last_name              address
0      CUST001       John       Doe    1 Main St, City 0
1      CUST002       Jane     Smith   88 Main St, City 1
2      CUST003       Jane     Smith   80 Main St, City 2
3      CUST004       Jane     Smith   32 Main St, City 3
4      CUST005       John       Doe   81 Main St, City 4
5      CUST006       Jane     Smith    8 Main St, City 5
6      CUST007       Jane     Smith   64 Main St, City 6
7      CUST008       Jane     Smith   98 Main St, City 7
8      CUST009       John       Doe   52 Main St, City 8
9      CUST010       Jane     Smith   96 Main St, City 9
10     CUST011       Jane     Smith  27 Main St, City 10
11     CUST012       Jane     Smith  80 Main St, City 11
12     CUST013       John       Doe  69 Main St, City 12
13     CUST014       Jane     Smith  44 Main St, City 13
14     CUST015       Jane     Smith  90 Main St, City 14
15     CUST016       Jane     Smith  92 Main St, City 15
16     CUST0

In [20]:
# Create an SQLite database in memory
conn = sqlite3.connect(":memory:")

# Load DataFrames into the SQLite database
customers_df.to_sql("CUSTOMERS", conn, index=False, if_exists="replace")
items_df.to_sql("ITEMS", conn, index=False, if_exists="replace")
sales_df.to_sql("SALES", conn, index=False, if_exists="replace")


50

Pull total number of orders that were completed on 18th March 2023 with the first name ‘John’ and last name Doe’


In [17]:
# Define and execute the query
query1 = """
SELECT COUNT(*) AS total_orders_john_doe
FROM SALES S
JOIN CUSTOMERS C ON S.customer_id = C.customer_id
WHERE S.date = '2023-03-18'
  AND C.first_name = 'John'
  AND C.last_name = 'Doe';
"""
result1 = pd.read_sql(query1, conn)
print("Total orders by John Doe on 18th March 2023:", result1['total_orders_john_doe'][0])


Total orders by John Doe on 18th March 2023: 0


Pull total number of customers that purchased in January 2023 and the average amount spend per customer

In [33]:
print(sales_df.head())
print(sales_df.columns)


         date order_id  item_id customer_id  quantity  revenue
0  2022-01-12   ORD001  ITEM002     CUST008         2    38.06
1  2022-01-20   ORD002  ITEM007     CUST001         2   100.44
2  2023-01-27   ORD003  ITEM004     CUST004         3    27.30
3  2023-02-06   ORD004  ITEM006     CUST015         2   106.24
4  2022-01-06   ORD005  ITEM003     CUST004         5   279.25
Index(['date', 'order_id', 'item_id', 'customer_id', 'quantity', 'revenue'], dtype='object')


In [23]:
# Query for total customers in January 2023 and average spend per customer
query2 = """
SELECT COUNT(DISTINCT customer_id) AS total_customers,
       AVG(total_spend) AS average_spend
FROM (
    SELECT customer_id, SUM(revenue) AS total_spend
    FROM SALES
    WHERE date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY customer_id
) AS customer_spend;
"""

# Run the query and display the result
result = pd.read_sql(query2, conn)
print("Total customers in January 2023:", result['total_customers'][0])
print("Average amount spent per customer in January 2023:", result['average_spend'][0])


Total customers in January 2023: 4
Average amount spent per customer in January 2023: 115.345


Pull the departments that generated less than $600 in 2022


In [27]:
query3 = """
SELECT I.department, SUM(S.revenue) AS total_revenue
FROM SALES S
JOIN ITEMS I ON S.item_id = I.item_id
WHERE strftime('%Y', S.date) = '2022'
GROUP BY I.department
HAVING total_revenue < 600;

"""

result = pd.read_sql(query3, conn)
print("Departments generating less than $600 in 2022:")
print(result)


Departments generating less than $600 in 2022:
Empty DataFrame
Columns: [department, total_revenue]
Index: []


What is the most and least revenue we have generated by an order


In [28]:
max_revenue_query = "SELECT MAX(revenue) AS max_revenue FROM SALES;"
min_revenue_query = "SELECT MIN(revenue) AS min_revenue FROM SALES;"

max_revenue_result = pd.read_sql(max_revenue_query, conn)
print("Maximum revenue generated by a single order:", max_revenue_result['max_revenue'][0])

min_revenue_result = pd.read_sql(min_revenue_query, conn)
print("Minimum revenue generated by a single order:", min_revenue_result['min_revenue'][0])


Maximum revenue generated by a single order: 359.4
Minimum revenue generated by a single order: 9.1


What were the orders that were purchased in our most lucrative order


In [38]:
max_revenue_order_query = """
SELECT order_id, SUM(revenue) AS total_revenue
FROM SALES
GROUP BY order_id
ORDER BY total_revenue DESC
LIMIT 1;
"""

most_lucrative_order_result = pd.read_sql(max_revenue_order_query, conn)

if not most_lucrative_order_result.empty:
    most_lucrative_order_id = most_lucrative_order_result['order_id'][0]
    print("Most lucrative order ID:", most_lucrative_order_id)
    print("Total revenue of the most lucrative order:", most_lucrative_order_result['total_revenue'][0])

    # Step 4: Fetch details of the most lucrative order
    order_details_query = f"SELECT * FROM SALES WHERE order_id = '{most_lucrative_order_id}';"
    order_details_result = pd.read_sql(order_details_query, conn)

    print("Details of the most lucrative order:")
    print(order_details_result)
else:
    print("No orders found.")


Most lucrative order ID: ORD037
Total revenue of the most lucrative order: 359.4
Details of the most lucrative order:
         date order_id  item_id customer_id  quantity  revenue
0  2022-05-03   ORD037  ITEM009     CUST001         5    359.4
